In [1]:
import pandas as pd
import boto3
import json
import os
from datetime import datetime

In [2]:
print(f"Início do salvamento: {datetime.now()}")

Início do salvamento: 2025-07-18 13:27:39.066049


In [3]:
# Carregar configurações e resultados
with open('config.json') as f:
    config = json.load(f)
BUCKET     = config['bucket_name']
RAW_PREF   = config['pasta_raw']
PROC_PREF  = config['pasta_processed']
ARCH_PREF  = config['pasta_archive']

# Carregar métricas completas
df_meta = pd.read_csv('metricas_completas.csv')
print(f"Linhas carregadas: {len(df_meta)} linhas")

# Extrair ID do experimento (único ou usar o primeiro)
id_experimento = df_meta['id_experimento'].iloc[0]

Linhas carregadas: 15 linhas


In [4]:
# Conectar S3 e upload métricas
s3 = boto3.client('s3')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Gerar nome seguindo o padrão
nome_meta = f"{id_experimento}_metricas_{timestamp}.csv"
key_meta  = f"{PROC_PREF}{nome_meta}"

# Salvar localmente e enviar
df_meta.to_csv(f"/tmp/{nome_meta}", index=False)
s3.upload_file(f"/tmp/{nome_meta}", BUCKET, key_meta)
print(f"✓ Métricas enviadas para s3://{BUCKET}/{key_meta}")

✓ Métricas enviadas para s3://experimento-lucas-barbosa/processed/exp_005_metricas_20250718_132739.csv


In [5]:
# Salvar log de execução
log = {
    'timestamp': datetime.now().isoformat(),
    'metrics_key': key_meta
}
log_key = f"logs/processamento_{id_experimento}_{timestamp}.json"
s3.put_object(Bucket=BUCKET, Key=log_key, Body=json.dumps(log, indent=2))
print(f"✓ Log salvo em s3://{BUCKET}/{log_key}")

✓ Log salvo em s3://experimento-lucas-barbosa/logs/processamento_exp_005_20250718_132739.json
